# Lab 11: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [ ]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2025-04-23 15:07:50,729 | INFO     |: Logger demo_notebook.log already initialized.
2025-04-23 15:07:50,730 | INFO     |: GUI is already running. Shutdown notebook to force restart the GUI.


In [96]:
# STOP_PLOTTER()
# Start the plotter
START_PLOTTER()

2025-04-23 16:10:20,999 | INFO     |: Creating New Plotter Process


# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [ ]:
import time
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    async def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        # packet should be in the form of "yaw_control?|target_yaw_value|max_clockwise_speed|min_clockwise_speed|max_counter_clockwise_speed|min_counter_clockwise_speed|P_GAIN|D_GAIN|alpha"
        for i in range(0, 24):
            angle= str(15*i)
            self.ble.send_command(CMD.YAW_CONTROL, "1|"+ angle + "|250|40|250|60|11|0.7|0.1")
            time.sleep(1)
        self.ble.send_command(CMD.YAW_CONTROL, "0|0|250|120|0|120|8|0.7|0.1")

        control_timestamps = []
        yaw = []
        distance = []
        distance_2 = []
        motor_input = []

        def extract_control_data(uuid, byte_array):
            string = byte_array.decode("utf-8") 
            split_str = string.split("|")
            control_timestamps.append(int(split_str[0]))
            yaw.append(float(split_str[1]))
            distance.append(int(split_str[2]))
            distance_2.append(int(split_str[3]))
            motor_input.append(float(split_str[4]))
    
        self.ble.start_notify(self.ble.uuid['RX_STRING'], extract_control_data)
        self.ble.send_command(CMD.SEND_YAW_MAPPING_DATA, "")

        async def sleep_for_5_secs():
            await asyncio.sleep(5)

        await sleep_for_5_secs()

        self.ble.stop_notify(self.ble.uuid['RX_STRING'])

        np_meters = np.array(distance) / 1000.0
        return np_meters[np.newaxis].T, np.array([])



In [90]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2025-04-23 16:08:49,881 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:81:80:25:12:64
2025-04-23 16:08:56,578 | INFO     |: Connected to c0:81:80:25:12:64


In [91]:
ble.send_command(CMD.GET_TOF1, "")
distance1 = int(ble.receive_string(ble.uuid['RX_STRING']))
print(f"Measured distance: {distance1}")


ble.send_command(CMD.GET_TOF2, "")
distance1 = int(ble.receive_string(ble.uuid['RX_STRING']))
print(f"Measured distance: {distance1}")

Measured distance: 1306
Measured distance: 442


In [97]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2025-04-23 16:10:29,095 | INFO     |:  | Number of observations per grid cell: 24
2025-04-23 16:10:29,097 | INFO     |:  | Precaching Views...


c:\Users\zhang\Classes\ECE_4160\ece4160\code\lab11\FastRobots-sim-release-main\localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2025-04-23 16:10:33,817 | INFO     |:  | Precaching Time: 4.718 secs
2025-04-23 16:10:33,818 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-23 16:10:33,819 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


2025-04-23 16:13:37,615 | INFO     |: Disconnected from C0:81:80:25:12:64


# Run an update step of the Bayes Filter

In [93]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
await loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_odom(current_odom[0], current_odom[1])

2025-04-23 16:09:04,483 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-23 16:09:04,487 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[3.881, 10.134, 30.877, 41.673, 58.447, 71.43, 90.527, 108.656, 119.723, 132.999, 152.425, 164.438, 177.689, 199.963, 213.434, 224.877, 236.001, 257.413, 266.074, 280.05, 300.832, 312.887, 329.435, 344.003]
[1326, 1573, 1591, 1760, 1940, 1625, 1355, 659, 668, 1344, 1452, 2168, 2009, 720, 690, 760, 1000, 1643, 1493, 1413, 1463, 1522, 1234, 924]
2025-04-23 16:09:36,669 | INFO     |: Update Step
2025-04-23 16:09:36,671 | INFO     |:      | Update Time: 0.001 secs
2025-04-23 16:09:36,673 | INFO     |: Bel index     : (np.int64(6), np.int64(4), np.int64(16)) with prob = 0.9999998
2025-04-23 16:09:36,673 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-04-23 16:09:36,674 | INFO     |: Belief        : (0.305, 0.000, 150.000)


In [89]:
cmdr.plot_gt(0,0.9914) # 0, 3 = 0, 0.9144
# cmdr.plot_gt(1.524,0.9914) # 5, 3 = 1.524, 0.9144
# cmdr.plot_gt(1.524,-0.9914) # 5, -3 = 1.524, -0.9144
# cmdr.plot_gt(-0.9144,-0.6096) # -3,-2 = -0.9144, -0.6096

2025-04-23 16:08:44,550 | INFO     |: Disconnected from C0:81:80:25:12:64
